import libraries

In [233]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

read the data

In [234]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
Y = dataset.iloc[:, -1].values
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

encoding categorical values with label encoder

In [235]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
X[:, 2] = encoder.fit_transform(X[:, 2])
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

encode categorical values with one hot encoder

In [236]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

split the data

In [237]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
X_train

array([[0.0, 0.0, 1.0, ..., 1, 0, 163830.64],
       [0.0, 1.0, 0.0, ..., 1, 1, 57098.0],
       [1.0, 0.0, 0.0, ..., 1, 0, 185630.76],
       ...,
       [1.0, 0.0, 0.0, ..., 1, 0, 181429.87],
       [0.0, 0.0, 1.0, ..., 1, 1, 148750.16],
       [0.0, 1.0, 0.0, ..., 1, 0, 118855.26]], dtype=object)

feature scaling

In [238]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

initialise ann

In [239]:
import tensorflow as tf
ann = tf.keras.models.Sequential()

add input layer and first hidden layer

In [240]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))


add second hidden layer

In [241]:
ann.add(tf.keras.layers.Dense(units = 5, activation = 'relu'))

add the output layer

In [242]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

compile the ann

In [243]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

train the ann

In [244]:
ann.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5287 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4632 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4433 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4343 - accuracy: 0.7959
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4285 - accuracy: 0.7961
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4236 - accuracy: 0.8015
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4197 - accuracy: 0.8114
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4158 - accuracy: 0.8171
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4107 - accuracy: 0.8223
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4046 - accura

predict a certain row

In [245]:
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

1/1 [==============================] - 0s 196ms/step


array([[0.03459853]], dtype=float32)

predict test set results

In [248]:
Y_predict = np.where(ann.predict(X_test) > 0.5, 1, 0)
np.concatenate((Y_predict.reshape(len(Y_predict), 1), Y_test.reshape(len(Y_test), 1)), axis = 1)

63/63 [==============================] - 0s 2ms/step


array([[0, 0],
       [0, 1],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

evaluate the model

In [250]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_predict, Y_test)
print('accuracy is =', accuracy)

accuracy is = 0.859
